In [ ]:
pip install PyPDF

In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
import os
from pinecone import Pinecone, ServerlessSpec

In [2]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','dcd526af-6f63-493b-8c71-7b2d43d75473')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV','gcp-starter')

In [3]:
pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)


In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
index_name = "mcahtbot" # put in the name of your pinecone index here

In [6]:
extracted_data = load_pdf("C:/Python/genai_medical-chatbot/data/")

# Create Text Chunks

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [9]:
#downloading the embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_results= embeddings.embed_query("UTS")
print("Leangth", len(query_results))

Leangth 384


In [ ]:
#query_results

In [13]:
index = pc.Index('mcahtbot')

In [24]:
#docsearch = pc.from_documents([t.page_content for t in text_chunks], embeddings, index_name=index_name)
docsearch = pc.from_documents([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: 'Pinecone' object has no attribute 'from_documents'

In [22]:
#Initializing the Pinecone
#pc(api_key=PINECONE_API_KEY,
 #             environment=PINECONE_API_ENV)

index_name="mchatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch = pc.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: 'Pinecone' object has no attribute 'from_texts'

In [23]:
#If we already have an index we can load it like this
docsearch = pc.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

AttributeError: 'Pinecone' object has no attribute 'from_existing_index'